Datasource: "Leafsnap: A Computer Vision System for Automatic Plant Species Identification,"  
Neeraj Kumar, Peter N. Belhumeur, Arijit Biswas, David W. Jacobs, W. John Kress, Ida C. Lopez, João V. B. Soares,  
Proceedings of the 12th European Conference on Computer Vision (ECCV),  
October 2012

In [1]:
tarurl = r'http://leafsnap.com/static/dataset/leafsnap-dataset.tar'

In [16]:
pwd

'C:\\Users\\User\\Documents\\GitHub\\datasets\\dataset\\images\\field'

In [2]:
%cd ../../datasets/

C:\Users\User\Documents\GitHub\datasets


In [ ]:
!curl -O $tarurl 

In [ ]:
!tar -xvf leafsnap-dataset.tar

## Imports

In [45]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import skimage
import os
import math
import random

In [18]:
%cd ./dataset/images/field/

[WinError 3] The system cannot find the path specified: './dataset/images/field/'
C:\Users\User\Documents\GitHub\datasets\dataset\images\field


In [19]:
tree_specs = os.popen('ls').read()
tree_specs = tree_specs.strip().split(sep='\n')

In [38]:
!mkdir test,train,vdate

A subdirectory or file test already exists.
Error occurred while processing: test.
A subdirectory or file train already exists.
Error occurred while processing: train.
A subdirectory or file vdate already exists.
Error occurred while processing: vdate.


In [ ]:
#move images into train, test, validate folders with subfolders for classes
test_ratio = .15
for spec in tree_specs:
    %cd ./$spec
    listing = os.popen('ls').read().strip().split(sep='\n')
    random.shuffle(listing) #inplace
    im_count = len(listing)
    test_size=val_size = math.ceil(test_ratio*im_count)
    train_size = im_count - (test_size + val_size)
    subfolders = ['train', 'test', 'vdate']
    for subfolder in subfolders:
        os.makedirs(os.path.join(os.path.dirname(os.getcwd()), subfolder, spec))
    for item in range(0, train_size):
        moved = listing.pop()
        !mv $moved ../train/$spec/$moved
    for item in range(0, test_size):
        moved = listing.pop()
        !mv $moved ../test/$spec/$moved
    for item in range(0, val_size):
        moved = listing.pop()
        !mv $moved ../vdate/$spec/$moved
    %cd ..
    !rmdir ./$spec

In [41]:
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)
vdate_datagen = ImageDataGenerator(rescale=1/255)

In [127]:
# Flow training images in batches of 128 using train_datagen generator
batch_size = 32
target_size = (256,256)
train_generator = train_datagen.flow_from_directory(
        r'./train/',  # This is the source directory for training images
        target_size=target_size, 
        batch_size=batch_size,
        color_mode='rgb',
        # Specify the classes explicitly
        classes = tree_specs,
        class_mode='categorical')

Found 1303 images belonging to 48 classes.


In [128]:
test_generator = test_datagen.flow_from_directory(
        r'./test/',
        target_size=target_size,
        batch_size=batch_size,
        color_mode='rgb',
        classes=tree_specs,
        class_mode='categorical')

Found 272 images belonging to 48 classes.


In [129]:
vdate_generator = vdate_datagen.flow_from_directory(
        r'./vdate/',
        target_size=target_size,
        batch_size=batch_size,
        color_mode='rgb',
        classes=tree_specs,
        class_mode='categorical')

Found 272 images belonging to 48 classes.


In [121]:
#shape of image array
train_generator.next()[0].shape

(32, 256, 256, 3)

# Modeling
### Sequential Model

In [161]:
sequential_mod = tf.keras.models.Sequential()
sequential_mod.add(Flatten(input_shape=(256,256,3)))
sequential_mod.add(Dense(128, activation='relu'))
sequential_mod.add(Dropout(.1))
sequential_mod.add(Dense(64, activation='relu'))
sequential_mod.add(Dropout(.1))
sequential_mod.add(Dense(48, activation='softmax'))

sequential_mod.summary()

sequential_mod.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 196608)            0         
_________________________________________________________________
dense_115 (Dense)            (None, 128)               25165952  
_________________________________________________________________
dropout_72 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_116 (Dense)            (None, 64)                8256      
_________________________________________________________________
dropout_73 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_117 (Dense)            (None, 48)                3120      
Total params: 25,177,328
Trainable params: 25,177,328
Non-trainable params: 0
_________________________________________

In [162]:
total_sample = train_generator.n

In [163]:
n_epochs = 10
history = sequential_mod.fit_generator(
        train_generator, 
        steps_per_epoch=int(total_sample/batch_size),  
        epochs=n_epochs,
        verbose=1)

Epoch 1/10


ValueError: Shape mismatch: The shape of labels (received (1536,)) should equal the shape of logits except for the last dimension (received (32, 48)).

### Convolution Model

In [170]:
conv_model = tf.keras.models.Sequential()
# First convolutional layer, note the specification of shape
conv_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(256,256,3)))
conv_model.add(Conv2D(64, (3, 3), activation='relu'))
conv_model.add(MaxPooling2D(pool_size=(2, 2)))
conv_model.add(Dropout(0.25))
conv_model.add(Flatten())
conv_model.add(Dense(128, activation='relu'))
conv_model.add(Dropout(0.5))
conv_model.add(Dense(48, activation='softmax'))

conv_model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
n_epochs = 10
conv_history = conv_model.fit_generator(
        train_generator, 
        steps_per_epoch=int(total_sample/batch_size),  
        epochs=n_epochs,
        verbose=1)

Epoch 1/10
 5/40 [==>...........................] - ETA: 4:32 - loss: 3.8651 - accuracy: 0.0312